In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [2]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band2
  atmpro = mls
  band = 2
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [-2, -2, 0]
  ng_refs = [2, 3, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 540
  vmin = 340
  w_diffuse = [(1.66, 1.66), (1.8, 1.8, 1.8), (1.8, 1.66, 1.45, 1.45, 1.45, 1.45)]
  wgt = [(0.6, 0.6), (0.6, 0.6, 0.6), (0.6, 0.6, 0.6, 0.6, 0.6, 0.6)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.118900                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.059806
0.000750      2.0  0.119818                  2.0  0.060829
0.001052      3.0  0.123301                  3.0  0.064053
0.001476      4.0  0.126628                  4.0  0.066967
0.002070      5.0  0.129548                  5.0  0.069850
0.002904      6.0  0.132105                  6.0  0.072666
0.004074      7.0  0.133894                  7.0  0.075352
0.005714      8.0  0.134868                  8.0  0.077839
0.008015      9.0  0.134709                  9.0  0.079997
0.011243     10.0  0.133098                 10.0  0.081696
0.015771     11.0  0.129701                 11.0  0.082727
0.022122     12.0  0.124028                 12.0  0.082699
0.031031     13.0  0.115601                 13.0  0.081190
0.043528     14.0  0.104960                 14.0  0.078552
0.061057     15.0  0.105091                 15.0  0.085199
0.085645     16.0  0.122690                 16.0  0.107270
0.120136     17.0  0.144798                 17.0  0.134367
0.168516     18.0  0.162870                 18.0  0.157478
0.236378     19.0  0.177022                 19.0  0.173836
0.331549     20.0  0.189644                 20.0  0.184535
0.465100     21.0  0.206032                 21.0  0.198289
0.652400     22.0  0.227466                 22.0  0.222734
0.915100     23.0  0.211525                 23.0  0.215990
1.283650     24.0  0.169448                 24.0  0.181173
1.800600     25.0  0.136518                 25.0  0.148718
2.525700     26.0  0.109344                 26.0  0.116662
3.542800     27.0  0.091800                 27.0  0.094671
4.969550     28.0  0.080593                 28.0  0.080924
6.970850     29.0  0.072098                 29.0  0.071705
9.778100     30.0  0.064670                 30.0  0.062678
13.715850    31.0  0.057762                 31.0  0.054277
19.239350    32.0  0.051205                 32.0  0.047923
26.987250    33.0  0.044974                 33.0  0.043186
37.855300    34.0  0.039068                 34.0  0.037589
53.100050    35.0  0.028492                 35.0  0.026180
73.887500    36.0  0.013776                 36.0  0.010894
97.662500    37.0  0.004116                 37.0  0.001453
121.437500   38.0  0.007674                 38.0  0.005539
145.212500   39.0  0.020978                 39.0  0.019426
168.987500   40.0  0.036753                 40.0  0.036232
192.762500   41.0  0.070259                 41.0  0.071824
216.537500   42.0  0.166599                 42.0  0.173062
240.312500   43.0  0.292787                 43.0  0.289024
264.087500   44.0  0.433296                 44.0  0.404622
287.862500   45.0  0.575545                 45.0  0.541738
311.637500   46.0  0.706863                 46.0  0.662569
335.412500   47.0  0.817635                 47.0  0.737444
359.187500   48.0  0.898222                 48.0  0.826393
382.962500   49.0  0.924448                 49.0  0.884811
406.737500   50.0  0.967756                 50.0  0.929752
430.512500   51.0  1.015138                 51.0  0.977595
454.287500   52.0  1.056514                 52.0  1.042309
478.062500   53.0  1.085549                 53.0  1.102965
501.837500   54.0  1.097525                 54.0  1.129287
525.612500   55.0  1.093802                 55.0  1.123795
549.387500   56.0  1.073029                 56.0  1.098695
573.162500   57.0  1.036783                 57.0  1.072267
596.937500   58.0  0.987585                 58.0  1.046449
620.712500   59.0  0.927590                 59.0  1.007601
644.487500   60.0  0.862490                 60.0  0.947282
668.262500   61.0  0.796197                 61.0  0.863710
692.037500   62.0  0.727999                 62.0  0.763310
715.812500   63.0  0.651818                 63.0  0.654721
739.587500   64.0  0.589967                 64.0  0.570567
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -61.195826   0.000000 -61.195826        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -60.873584   0.000003   
6.244000e-04 2     -61.195831   0.000014 -61.195817 -60.873589   0.000012   
8.759000e-04 3     -61.195833   0.000020 -61.195813 -60.873591   0.000016   
1.228600e-03 4     -61.195836   0.000028 -61.195808 -60.873594   0.000022   
1.723400e-03 5     -61.195841   0.000040 -61.195801 -60.873599   0.000030   
2.417400e-03 6     -61.195846   0.000056 -61.195790 -60.873605   0.000042   
3.390900e-03 7     -61.195855   0.000080 -61.195775 -60.873613   0.000059   
4.756500e-03 8     -61.195866   0.000113 -61.195753 -60.873625   0.000083   
6.672000e-03 9     -61.195882   0.000159 -61.195723 -60.873641   0.000117   
9.358900e-03 10    -61.195904   0.000224 -61.195680 -60.873664   0.000165   
1.312780e-02 11    -61.195935   0.000315 -61.195620 -60.873696   0.000234   
1.841450e-02 12    -61.195979   0.000439 -61.195539 -60.873741   0.000331   
2.583020e-02 13    -61.196040   0.000610 -61.195430 -60.873805   0.000467   
3.623230e-02 14    -61.196129   0.000841 -61.195288 -60.873895   0.000657   
5.082340e-02 15    -61.196257   0.001151 -61.195106 -60.874024   0.000922   
7.129060e-02 16    -61.196439   0.001588 -61.194852 -60.874206   0.001310   
1.000000e-01 17    -61.196684   0.002250 -61.194434 -60.874450   0.001919   
1.402710e-01 18    -61.196998   0.003255 -61.193744 -60.874761   0.002871   
1.967600e-01 19    -61.197384   0.004731 -61.192654 -60.875142   0.004306   
2.759970e-01 20    -61.197828   0.006836 -61.190992 -60.875574   0.006370   
3.871000e-01 21    -61.198272   0.009775 -61.188496 -60.875995   0.009220   
5.431000e-01 22    -61.198532   0.013843 -61.184689 -60.876215   0.013105   
7.617000e-01 23    -61.198134   0.019335 -61.178799 -60.875748   0.018405   
1.068500e+00 24    -61.196878   0.025766 -61.171111 -60.874379   0.024887   
1.498800e+00 25    -61.194983   0.032509 -61.162474 -60.872330   0.032074   
2.102400e+00 26    -61.192438   0.039725 -61.152713 -60.869582   0.039960   
2.949000e+00 27    -61.189387   0.047639 -61.141747 -60.866280   0.048359   
4.136600e+00 28    -61.185806   0.056973 -61.128833 -60.862377   0.057776   
5.802500e+00 29    -61.181559   0.068630 -61.112929 -60.857729   0.069099   
8.139200e+00 30    -61.176500   0.083527 -61.092972 -60.852150   0.083370   
1.141700e+01 31    -61.170499   0.102636 -61.067863 -60.845510   0.101069   
1.601470e+01 32    -61.163494   0.127090 -61.036404 -60.837739   0.122862   
2.246400e+01 33    -61.155565   0.158280 -60.997285 -60.828980   0.150718   
3.151050e+01 34    -61.146970   0.197881 -60.949089 -60.819625   0.187647   
4.420010e+01 35    -61.137901   0.247538 -60.890363 -60.810123   0.234654   
6.200000e+01 36    -61.134429   0.304142 -60.830287 -60.807316   0.287054   
8.577500e+01 37    -61.152400   0.360910 -60.791490 -60.828384   0.338806   
1.095500e+02 38    -61.192700   0.412804 -60.779897 -60.873012   0.387526   
1.333250e+02 39    -61.244101   0.485817 -60.758283 -60.928392   0.458507   
1.571000e+02 40    -61.296048   0.596845 -60.699203 -60.983665   0.568496   
1.808750e+02 41    -61.349486   0.753792 -60.595694 -61.039905   0.726786   
2.046500e+02 42    -61.415655   1.017832 -60.397823 -61.109481   0.998663   
2.284250e+02 43    -61.530077   1.601450 -59.928627 -61.231099   1.607729   
2.522000e+02 44    -61.706041   2.601997 -59.104044 -61.418196   2.608894   
2.759750e+02 45    -61.954737   4.070990 -57.883746 -61.680181   4.010540   
2.997500e+02 46    -62.284615   6.021788 -56.262828 -62.023553   5.879776   
3.235250e+02 47    -62.701482   8.429407 -54.272076 -62.452368   8.174789   
3.473000e+02 48    -63.209652  11.240297 -51.969355 -62.972769  10.772283   
3.710750e+02 49    -63

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')